In [1]:
# import neccessary libraries
import numpy as np
import random
import sympy as sym
from joblib import Parallel, delayed
from sympy import *
from sympy import symbols
from sympy.plotting import plot, plot3d
from datetime import datetime
from scipy.stats import beta
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

In [2]:
# function to count the number of the infected neighbores of i at t:
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [3]:
# Function to obtain the very initial sample of X, using forwad sampling:
def Forward_Sampling(T,n,G,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4] 
    x=int((1-P)*n)
    X=np.zeros((n,T))  
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    infR=CNbr(G,X,n,T)
    for t in range(T-1):
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c
                p1=alpha_+beta_*c
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 
    return X

In [69]:
def Sample_hidden_state(T,n,X,G,Y,param,t):
    
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]
    
    pow_stay_healthy,pow_become_infected,pow_recovery,pow_stay_infected=0,0,0,0
    
    if (t==0):
        c=G[t].dot(X.T[t])
    else:    
        c=G[t-1].dot(X.T[t-1])
        
    for i in range(n):
        
        alpha_=param[0]
        beta_=param[1]
        gama_=param[2]
        theta_0_=param[3]
        thata_1_=param[4] 
    
        pow0=np.count_nonzero(Y[i,t] == 0)  
        pow1=np.count_nonzero(Y[i,t] == 1)
    
        p0=(1-theta_0_)**pow0
        p1=(1-theta_1_)**pow0
    
        p0=(theta_0_)**pow1
        p1=theta_1_**pow1
        
        if (X[i,t]==1)&(t==0):
            c0=c[i]-1
            c1=c[i]
        else:
            c0=c[i]
            c1=c[i]+1
        
                
        stay_healthy0= 1-alpha_-beta_*c0
        become_infected0=alpha_+beta_*c0
    
        stay_healthy1=1-alpha_-beta_*c1
        become_infected1=alpha_+beta_*c1
        
        if t==T-1:
            if X[i,t-1]==0:
                p0=p0*stay_healthy0
                p1=p1*become_infected0
            else:
                p0=p0*gama_
                p1=p1*(1-gama_) 
        else:        
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[0][i],state_transition)
    
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
    
            p0=P*p0*(stay_healthy0)**pow_stay_healthy*(become_infected0)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            p1=(1-P)*p1*(stay_healthy1)**pow_stay_healthy*(become_infected1)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            
        if p0+p1==0:
            l=0.5
        else:
            l=p1/(p0+p1)
        if (l>1 )| (l==0) | (l<0 ):
            print(i,t,p0,p1,stay_healthy0,become_infected0,c0)
        X[i,0]=np.random.binomial( 1,  l,size=None)    
        
    return X    
    

In [5]:
# Gibbs sampling to obtain X, as new sample of posterior distribution:
def Calculate_X(K,T,n,X,G,Y,param):

    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]
    
    for k in range(K):
        for t in range(T):
            X=Sample_hidden_state(T,n,X,G,Y,param,t)
    return X                

In [6]:
# function to sample new parameters and update parameters:
def Params(R,G,X,n,T,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]    
   
    TP=np.sum(np.multiply(X,Y))
    FP=n*T-np.count_nonzero(X-Y+1)
    
    infR=np.array(CNbr(G,X,n,T))
    
    alpha_=Sample_alpha(a_alpha + n*T- np.count_nonzero(R) , b_alpha - np.count_nonzero(X)+ np.count_nonzero(R))
    beta_=Sample_beta(a_beta + n*T-np.count_nonzero(R-2) , b_beta +np.sum(np.multiply((1-X),infR))-n*T+np.count_nonzero(R-2))
    gama_=Sample_gama(a_gama +(T-1)*n-np.count_nonzero(X[:,:-1]-X[:,1:]-1), b_gama+np.sum(X)-(T-1)*n+np.count_nonzero(X[:,:-1]-X[:,1:]-1))
    theta_0_=Sample_theta0( a_teta0+FP,b_teta0+n*T-np.count_nonzero(X)-FP)
    theta_1_=Sample_theta1( a_teta1+TP,b_teta1+np.count_nonzero(X)-TP)
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infr=int(infR[i,t])
            pr_a=alpha_/(alpha_+beta_*infr)
            pr_b=beta_/(alpha_+beta_*infr)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

    return alpha_,beta_,gama_,theta_0_,theta_1_,R

# function to sample from beta distribution


In [7]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=1000):
        if (i>0.001)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        


In [8]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=1000):
        if (i>0.0001)&(i<0.051):
            beta_=round(i,4)
            break
    return beta_        


In [9]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=1000):
        if (i>0.1)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  


In [10]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        if (i>0.001)&(i<0.3):
            theta_0_=round(i,3)
            break
    return theta_0_  


In [11]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=10000):
        if i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  


In [45]:
# initialize parameters for beta distributions:
a_alpha=1
b_alpha=10
a_beta=0.1
b_beta=1
a_gama=7
b_gama=7
a_teta0=1
b_teta0=5
a_teta1=5
b_teta1=.1

In [46]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)
p_=.8

In [60]:
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])


In [61]:
# Function to generates Synthetic dataset
def Synthetic_Data(n,T,y,params):
    alpha_,beta_,gama_,theta_0_,theta_1_,p_=params[0],params[1],params[2],params[3],params[4],params[5]
    x=int((1-p_)*n)

    X=np.zeros((n,T))
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    # Random social network
    G=[]
    for j in range(T):
        g=np.identity(n)
        for i in range(n):
            inx=np.random.choice(range(i,n), y)
            g[i,inx]=1  
            g[inx,i]=1
        G.append(g)
    G=np.array(G)
    infR=CNbr(G,X,n,T)
    # Synthetize X, using params,G and transition probability:
    for t in range(T-1):
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c
                p1=alpha_+beta_*c
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 

    # Synthetize Y, using params,G, X, emission probability:
    Y=np.zeros((n,T))
    for t in range(T):
        for i in range(n):
            if X[i,t]==0:
                Y[i,t]=np.random.binomial( 1, theta_0_,size=None) 
            else:
                Y[i,t]=np.random.binomial( 1, theta_1_,size=None) 
    return G,Y,X            

In [62]:
n,T,y=100,100,10
synthetic_data=Synthetic_Data(n,T,y,params[-1])
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

In [63]:
# Save true value of X in Z, as the correct label of data:
Z=X
np.sum(Z)

895.0

In [64]:
np.sum(X)

895.0

In [65]:
params

[[0.051, 0.0191, 0.538, 0.019, 0.999, 0.8]]

In [66]:
np.sum(Y)

1075.0

In [70]:
# Run our MCEM algorithm to estimate X:
U=1
J=10
P=p_
X=Forward_Sampling(T,n,G,Y,param[-1])
X=Calculate_X(J,T,n,X,G,Y,param[-1])
winsound.Beep(freq, duration)

73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0
73 3 5e-324 0.0 0.9905 0.009500000000000001 1.0
73 4 5e-324 0.0 0.999 0.001 0.0


In [71]:
np.sum(X)

42.0

In [55]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [56]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,Y,param[-1])
    param=[prm[0]]
    X=Calculate_X(J,T,n,X,G,Y,param[-1])
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)            

******************* Iteration: 0 *****************************************************************************
Current Time is : 15:38:53


UnboundLocalError: local variable 'theta_1_' referenced before assignment

In [ ]:
# Compare X, Z and estimate accuracy of the model:

In [ ]:
# synthetized observation vectore:
Y_=np.zeros((n,T))
for t in range(T):
    for i in range(n):
        if X[i,t]==0:
            Y_[i,t]=np.random.binomial( 1, theta_0_,size=None) 
        else:
            Y_[i,t]=np.random.binomial( 1, theta_1_,size=None) 

In [ ]:
r=10
K=1000
J=10

# test the model with synthetized data, Y_,G:
for i in range(r):
    print("MCEM #iteration:",i)
    X=Forward_Sampling(T,n,G,Y_,params[-1])
    Q=0
    for j in range(J):
        print(" #Samples Generated in GibbsSampling:",j)
        X=Calculate_X(K,T,n,X,G,Y_,params[-1])
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time is :", current_time)
        print("Verctor of Health States:","\n",X)
        print(np.sum(X))
        Q=np.sum(np.array(transition(X,CNbr(G,X,n,T),n,T))+emission(X,Y_,n,T),axis=0)+Q    
    a=round(float(solve(simplify(Q.diff(theta_0)))[0]),3)
    b=round(float(solve(simplify(Q.diff(theta_1)))[0]),3)
    ga=round(float(solve(simplify(factor(Q.diff(gamma))))[0]),3)
    o=Q.subs(theta_0,a).subs(theta_1,b).subs(gamma,ga)
    o=expand_log(o, force=True)
    p_=round(float(solve(o.diff(pi))[0]),3)
    o=o.subs(pi,p_)
    alta=nsolve((o.diff(alpha),o.diff(beta)),(alpha,beta),(alpha_,beta_))
#******************************update parameters
    alpha_=round(alta[0],3)
    beta_=round(alta[1],3)
    gama_=ga
    theta_0_=a
    theta_1_=b
    params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    print("New parameters:",alpha_,beta_,gama_,theta_0_,theta_1_,p_)
    print("Verctor of Health States:","\n",X)

np.sum(X)

In [ ]:
np.size(X)

In [ ]:
np.sum(Y1)

In [ ]:
plot3d(o,(alpha,.061,.08),(beta,0.001999,0.04009))

In [ ]:
# Choose some params close to obtimized params:
alpha_= 0.05
beta_= 0.01
gama_= 0.3
theta_0_= 0.07
theta_1_= 0.8
p_= 0.8
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

In [ ]:
plot3d(o,(beta,0.001999,0.04009),(alpha,.061,.08))

In [ ]:
plot3d(o.diff(alpha),(beta,0.001999,0.04009),(alpha,.061,.08))

In [ ]:
plot3d(o.diff(beta),(beta,0.001999,0.04009),(alpha,.061,.08))

In [ ]:
plot3d(o,(beta,.00161,.01999908),(alpha,.06961,.079949799908))

In [ ]:
plot3d(o,(beta,0.0041999,0.04009),(alpha,.061,.08))

In [ ]:
plot3d(o,(alpha,-.000061,.18),(beta,-0.00000041999,0.0019))

In [ ]:
plot3d(o,(beta,-0.00000041999,0.0019),(alpha,-.000061,.18))

# ##############################################################################################################################

In [ ]:
# The following is for family test result problem:

In [ ]:
# Randomly initialize params:
number_families=30
alpha_=0.005
beta_=0.045 
gama_=0.3
theta_0_=0.01
theta_1_=0.9
p_=0.9

params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

# Synthetize data for famity test result problem

In [ ]:
# Synthetize the "Family" matrix, which denotes that each two individuals are in the same family or not:
a=list(range(n))
Family=np.identity(n)
for i in range(1,number_families):
    number_family_members=np.random.randint(2, 5)
    inx=random.sample(list(a), number_family_members)
    a=set(a).difference(inx)
    for j in inx:
        for k in inx:
            Family[j,k]=1
Family

In [ ]:
# Synthetize G,Y,X:
n,T,y=100,100,10
synthetic_data=Synthetic_Data(n,T,y,params[-1])
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

In [ ]:
# synthetize family test result matrix,YF:
unique_rows = np.unique(Family, axis=0)
unique_rows.shape
YF=np.zeros((n,T))
YF=np.dot(unique_rows,Y)
YF.shape

# Simulation data is ready now, YF,G,Family:

In [ ]:
# number of neighbore in family
neighbore_family=[]
for t in range(T):
    neighbore_family.append(np.dot(unique_rows,G[t]))
neighbore_family=np.array(neighbore_family)

neighbore_family.shape

In [ ]:
#number_infected_neighbores for each individual i at each time t:
number_infected_neighbore=[]
for t in range(T):
    number_infected_neighbore.append(np.dot(neighbore_family[t].T,YF.T[t].T))

number_infected_neighbore=np.array(number_infected_neighbore)
number_infected_neighbore.shape

In [ ]:
# number_infected_neighbores for each member of each family at each time::
infected_neighbore_of_family_members=[]
for t in range(T):
    infected_neighbore_of_family_members.append(np.multiply(unique_rows,number_infected_neighbore[t]))
infected_neighbore_of_family_members=np.array(infected_neighbore_of_family_members)  

infected_neighbore_of_family_members.shape

In [ ]:
# generate synthetized Y_, which is our sudo observation matrix:
Y_=np.zeros((n,T))
for i in range(number_families):
    for t in range(T):
        an_array=infected_neighbore_of_family_members[t][i]
        indx = np.argsort(an_array)[-2:]
        if len(indx)>0:
            Y_[indx,t]=1
Y_


np.sum(Y_)

#  Now we have sudo observation matrix Y_ and social network G, so  we do estimate X and parameters for these synthetic dataset:


In [ ]:
r=10
K=1000
J=1

# test the model with synthetized data, Y_,G:
for i in range(r):
    print("MCEM #iteration:",i)
    X=Forward_Sampling(T,n,G,Y_,params[-1])
    Q=0
    for j in range(J):
        print(" #Samples Generated in GibbsSampling:",j)
        X=Calculate_X(K,T,n,X,G,Y_,params[-1])
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time is :", current_time)
        print("Verctor of Health States:","\n",X)
        print(np.sum(X))
        Q=np.sum(np.array(transition(X,CNbr(G,X,n,T),n,T))+emission(X,Y_,n,T),axis=0)+Q    
    a=round(float(solve(simplify(Q.diff(theta_0)))[0]),3)
    b=round(float(solve(simplify(Q.diff(theta_1)))[0]),3)
    ga=round(float(solve(simplify(factor(Q.diff(gamma))))[0]),3)
    o=Q.subs(theta_0,a).subs(theta_1,b).subs(gamma,ga)
    o=expand_log(o, force=True)
    p_=round(float(solve(o.diff(pi))[0]),3)
    o=o.subs(pi,p_)
    alta=nsolve((o.diff(alpha),o.diff(beta)),(alpha,beta),(alpha_,beta_))
#******************************update parameters
    alpha_=round(alta[0],3)
    beta_=round(alta[1],3)
    gama_=ga
    theta_0_=a
    theta_1_=b
    params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    print("New parameters:",alpha_,beta_,gama_,theta_0_,theta_1_,p_)
    print("Verctor of Health States:","\n",X)

np.sum(X)

In [ ]:
params[-1]